In [1]:
import sqlite3
import requests
from tqdm.auto import tqdm
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

stops = set(stopwords.words('russian') + ['это', 'весь'])

Откроем базу данных и создадим курсор.

In [2]:
con = sqlite3.connect('vk_api_pm.db')
cur = con.cursor()

Создадим таблицы в базу данных. Мне показалось, что с метаинформацией немного запутанно, не очень понятно, нужна ли метаинформация пользователя или комментария, поэтому я создаю три таблицы для экономии места: с комментариями, их айди и лемматизацией; с метаинформацией комментариев (айди, айди пользователя и количество ответов) и метаинформацией авторов комментариев (айди пользователя, дата рождения и город).

In [3]:
cur.execute("""
CREATE TABLE comments (
    comment_id INT, 
    text TEXT, 
    lemmatized_text TEXT,
    PRIMARY KEY (comment_id)
)
""")

In [4]:
cur.execute("""
CREATE TABLE comments_meta (
    comment_id INT, 
    user_id INT,
    comment_thread INT,
    PRIMARY KEY (comment_id)
)
""")

In [5]:
cur.execute("""
CREATE TABLE users (
    user_id INT, 
    city TEXT,
    sex TEXT,
    PRIMARY KEY (user_id)
)
""")

Запишем токен (здесь скрыт) и версию.

In [13]:
TOKEN = ""
VERSION = "5.130"

In [14]:
wall_get_url = "https://api.vk.com/method/wall.get"
get_comments_url = "https://api.vk.com/method/wall.getComments"
users_get_url = "https://api.vk.com/method/users.get"

Скачаем информацию 1800 постов из одной группы ("поиск мемов по описанию"), чтобы потом доставать комментарии.

In [15]:
all_the_data = []
offs = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700]
for off in offs:
    data = requests.get(
        wall_get_url, 
        params={
            "owner_id": -166124324,
            "count": 100,
            "v": VERSION,
            "access_token": TOKEN,
            "offset": off, 
        }
    ).json()
    all_the_data.append(data)

Создадим список айди этих постов и на всякий случай проверим количество.

In [16]:
post_ids = []
for data in all_the_data:
    for da in data['response']['items']:
        post_ids.append(da['id'])
    
print(len(post_ids))

1800


Достанем всю информацию обо всех комментариях к этим постам.

In [ ]:
all_coms = []
for post_id in tqdm(post_ids):
    data_com = requests.get(
        get_comments_url, 
        params={
            "owner_id": -166124324,
            "post_id": post_id,
            "v": VERSION,
            "access_token": TOKEN
        }
    ).json()
    all_coms.append(data_com)

Достанем id всех комментаторов, а потом подсчитаем количество комментариев у каждого и выберем сотню тех, кто написал 10+ (нужно не забыть убрать комментарии сообщества).

In [ ]:
ids = []
for a in all_coms:
    for b in a['response']['items']:
        if 'from_id' in b:
            ids.append(b['from_id'])
        else:
            pass
dicti = Counter(ids)
needed_ids = []
for d in dicti.most_common(101):
    if str(d[0]).startswith('-'):
        pass
    else:
        needed_ids.append(d[0])
print(needed_ids)

Сначала из всех комментариев к постам достанем те, которые написаны нужными пользователями (сотня тех, у кого есть хотя бы 10 комментариев), а потом достанем 10 комментариев от каждого из ста пользователей.

In [ ]:
all_n_coms = []
for a in all_coms:
    for b in a['response']['items']:
        if 'from_id' in b:
            if b['from_id'] in needed_ids:
                all_n_coms.append(b)

set_u_id = []
set_c_id = []
set_c_thread = []
set_com = []
for i in needed_ids:
    counter = 0
    for n in all_n_coms:
        if n['from_id'] == i:
            counter += 1
            set_c_id.append(n['id'])
            set_u_id.append(i)
            set_c_thread.append(n['thread']['count'])
            set_com.append(n['text'])
            
            if counter == 10:
                break
data = {'comment_id': set_c_id, 'user_id': set_u_id, 'comment_answers': set_c_thread, 'comment_text': set_com}
full_df = pd.DataFrame(data)
full_df

Напишем функцию для лемматизации текста, а потом добавим новую колонку к таблице full_df

In [ ]:
def lemmatize_text(original_text):
    lem_text = str()
    original_text.replace(',', '')
    original_text.replace(')', '')
    original_text.replace('(', '')
    original_text.replace('.', '')
    original_text.replace('\n', '')
    words = original_text.split(' ')
    for word in words:
        if word not in stops:
            ana = morph.parse(word)
            lemma = ana[0].normal_form
            lem_text = lem_text + lemma + ' ' 
    return lem_text

In [ ]:
full_df['lemmatized_text'] = full_df['comment_text'].apply(lemmatize_text)
full_df

Создадим из большой таблицы две разные - одну с метаинформацией, другую с текстами, а потом добавим их в базу данных.

In [ ]:
df1 = full_df.drop(['user_id', 'comment_answers'], axis=1)

In [ ]:
df2 = full_df.drop(['comment_text', 'lemmatized_text'], axis=1)

In [ ]:
to_db = list(df1.values.tolist()) 
cur.executemany("INSERT INTO comments VALUES (?, ?, ?)", to_db)
con.commit()

In [ ]:
t_db = list(df2.values.tolist()) 
cur.executemany("INSERT INTO comments_meta VALUES (?, ?, ?)", t_db)
con.commit()

Достанем всю информацию о сотне пользователей с нужными id и добавим её в список user_infos

In [ ]:
user_infos = []
for user_id in needed_ids:
    user_info = requests.get(
        users_get_url,
        params={
            'user_ids': user_id,
            'fields': 'city,sex',
            'access_token': TOKEN,
            'v': VERSION
        }
    ).json()
    user_infos.append(user_info)

Достанем только необходимую информацию о каждом пользователе - id, пол и город.

In [ ]:
user_ids = []
sex = []
city = []
for u_i in user_infos:
    user_ids.append(u_i['response'][0]['id'])
    if 'city' in u_i['response'][0]:
        city.append(u_i['response'][0]['city']['title'])
    else:
        city.append('0')  
    if u_i['response'][0]['sex'] == 1:
        sex.append('female')
    elif u_i['response'][0]['sex'] == 2:
        sex.append('male')
    
                
user_data = {'user_id': user_ids,  'city': city, 'sex': sex}
user_df = pd.DataFrame(user_data)
user_df

Добавим таблицу с информацией  пользователях в базу данных. 

In [ ]:
t_d = list(user_df.values.tolist()) 
cur.executemany("INSERT INTO users VALUES (?, ?, ?)", t_d)
con.commit()